# Loan data processing project

In [1]:
import numpy as np
np.set_printoptions(suppress = True, linewidth = 100, precision = 2)

Below is a helper function for creating checkpoints of the data, we shall need it later on.

In [2]:
def checkpoint(file_name, checkpoint_header,checkpoint_data):
    np.savez(file_name, header = checkpoint_header, data = checkpoint_data)
    checkpoint_variable = np.load(file_name + ".npz")
    return(checkpoint_variable)

## Loading and inspecting the data

In [3]:
raw_data = np.genfromtxt(r"loan-data.csv", delimiter = ';', skip_header = 1, autostrip = True, encoding='cp855')
print(raw_data)
print('\nShape:',raw_data.shape)

[[48010226.           nan    35000.   ...         nan         nan     9452.96]
 [57693261.           nan    30000.   ...         nan         nan     4679.7 ]
 [59432726.           nan    15000.   ...         nan         nan     1969.83]
 ...
 [50415990.           nan    10000.   ...         nan         nan     2185.64]
 [46154151.           nan         nan ...         nan         nan     3199.4 ]
 [66055249.           nan    10000.   ...         nan         nan      301.9 ]]

Shape: (10000, 14)


The data has 10000 rows and 14 columns. A notable aspect is that there is a lot of missing data (`nan` values). 

In [4]:
print(np.isnan(raw_data).sum())

88005


Let us collect minimum, maximum, and mean values from all columns.

In [5]:
temporary_fill = np.nanmax(raw_data) + 1 
temporary_mean = np.nanmean(raw_data, axis = 0)
temporary_stats = np.array([np.nanmin(raw_data, axis = 0), temporary_mean, np.nanmax(raw_data, axis = 0)])
print(temporary_stats)
print(temporary_stats.shape)

[[  373332.           nan     1000.           nan     1000.           nan        6.         31.42
          nan         nan         nan         nan         nan        0.  ]
 [54015809.19         nan    15273.46         nan    15311.04         nan       16.62      440.92
          nan         nan         nan         nan         nan     3143.85]
 [68616519.           nan    35000.           nan    35000.           nan       28.99     1372.97
          nan         nan         nan         nan         nan    41913.62]]
(3, 14)


/var/folders/14/kmkw5ndj0kq_53pqzfg6fv2m0000gn/T/ipykernel_35611/2952517997.py:2: RuntimeWarning: Mean of empty slice
  temporary_mean = np.nanmean(raw_data, axis = 0)
/var/folders/14/kmkw5ndj0kq_53pqzfg6fv2m0000gn/T/ipykernel_35611/2952517997.py:3: RuntimeWarning: All-NaN slice encountered
  temporary_stats = np.array([np.nanmin(raw_data, axis = 0), temporary_mean, np.nanmax(raw_data, axis = 0)])


The data is heterogeneous, it contains both characters and numbers. This partly introduces `nan` values. So, we split the data, for convenience.

In [6]:
columns_strings = np.argwhere(np.isnan(temporary_mean)).squeeze()
print(columns_strings)
columns_numeric = np.argwhere(np.isnan(temporary_mean) == False).squeeze()
print(columns_numeric)

[ 1  3  5  8  9 10 11 12]
[ 0  2  4  6  7 13]


Let us re-import the data correctly. First, string columns.

In [7]:
loan_data_strings = np.genfromtxt(r"loan-data.csv", 
                                  delimiter = ';', 
                                  skip_header = 1, 
                                  autostrip = True, 
                                  usecols = columns_strings,
                                  dtype = np.str,
                                  encoding='cp855')
print(loan_data_strings)
print('\nShape:',loan_data_strings.shape)

[['May-15' 'Current' '36 months' ... 'Verified'
  'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226' 'CA']
 ['' 'Current' '36 months' ... 'Source Verified'
  'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261' 'NY']
 ['Sep-15' 'Current' '36 months' ... 'Verified'
  'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726' 'PA']
 ...
 ['Jun-15' 'Current' '36 months' ... 'Source Verified'
  'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990' 'CA']
 ['Apr-15' 'Current' '36 months' ... 'Source Verified'
  'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151' 'OH']
 ['Dec-15' 'Current' '36 months' ... ''
  'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249' 'IL']]

Shape: (10000, 8)


/var/folders/14/kmkw5ndj0kq_53pqzfg6fv2m0000gn/T/ipykernel_35611/4026404118.py:6: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.str,


Second, numeric columns.

In [8]:
loan_data_numeric = np.genfromtxt(r"loan-data.csv", 
                                  delimiter = ';', 
                                  skip_header = 1, 
                                  autostrip = True, 
                                  usecols = columns_numeric,
                                  filling_values = temporary_fill,
                                  encoding='cp855')
print(loan_data_numeric)
print('\nShape:',loan_data_numeric.shape)

[[48010226.      35000.      35000.         13.33     1184.86     9452.96]
 [57693261.      30000.      30000.   68616520.        938.57     4679.7 ]
 [59432726.      15000.      15000.   68616520.        494.86     1969.83]
 ...
 [50415990.      10000.      10000.   68616520.   68616520.       2185.64]
 [46154151.   68616520.      10000.         16.55      354.3      3199.4 ]
 [66055249.      10000.      10000.   68616520.        309.97      301.9 ]]

Shape: (10000, 6)


Next, let us import the header (column names).

In [9]:
header_full = np.genfromtxt(r"loan-data.csv", 
                              delimiter = ';',  
                              autostrip = True, 
                              skip_footer = raw_data.shape[0],
                              dtype = np.str,
                              encoding='cp855')
print(header_full)

['id' 'issue_d' 'loan_amnt' 'loan_status' 'funded_amnt' 'term' 'int_rate' 'installment' 'grade'
 'sub_grade' 'verification_status' 'url' 'addr_state' 'total_pymnt']


/var/folders/14/kmkw5ndj0kq_53pqzfg6fv2m0000gn/T/ipykernel_35611/3802774864.py:5: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.str,


We also may need headers for string an numeric subsets of the data.

In [10]:
header_strings, header_numeric = header_full[columns_strings], header_full[columns_numeric]
print('String columns:',header_strings)
print('Numeric columns:',header_numeric)

String columns: ['issue_d' 'loan_status' 'term' 'grade' 'sub_grade' 'verification_status' 'url' 'addr_state']
Numeric columns: ['id' 'loan_amnt' 'funded_amnt' 'int_rate' 'installment' 'total_pymnt']


## Manipulating Test Data: Issue Date

In [11]:
print(header_strings)
header_strings[0] = "issue_date"

print(loan_data_strings[:,0])

print(np.unique(loan_data_strings[:,0]))

loan_data_strings[:,0] = np.chararray.strip(loan_data_strings[:,0],"-15")

print(np.unique(loan_data_strings[:,0]))
months = np.array(['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

for i in range(13):
    loan_data_strings[:,0] = np.where(loan_data_strings[:,0] == months[i],
                                       i,
                                       loan_data_strings[:,0])
np.unique(loan_data_strings[:, 0])

['issue_d' 'loan_status' 'term' 'grade' 'sub_grade' 'verification_status' 'url' 'addr_state']
['May-15' '' 'Sep-15' ... 'Jun-15' 'Apr-15' 'Dec-15']
['' 'Apr-15' 'Aug-15' 'Dec-15' 'Feb-15' 'Jan-15' 'Jul-15' 'Jun-15' 'Mar-15' 'May-15' 'Nov-15'
 'Oct-15' 'Sep-15']
['' 'Apr' 'Aug' 'Dec' 'Feb' 'Jan' 'Jul' 'Jun' 'Mar' 'May' 'Nov' 'Oct' 'Sep']


array(['0', '1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='<U69')

## Manipulating Text Data: Loan Status

In [12]:
print(np.unique(loan_data_strings[:,1]))

bad_status = ['', 'Charged Off', 'Default', 'Late (31-120 days)']
loan_data_strings[:,1] = np.where(np.isin(loan_data_strings[:,1], bad_status),0,1)
print(np.unique(loan_data_strings[:,1]))

#Term
print(np.unique(loan_data_strings[:,2]))
loan_data_strings[:,2] = np.chararray.strip(loan_data_strings[:,2]," months")
print(loan_data_strings[:,2])
header_strings[2] = "term_months"
loan_data_strings[:,2] = np.where(loan_data_strings[:,2] == '',
                                 '60',
                                 loan_data_strings[:,2])

np.unique(loan_data_strings[:,2])

['' 'Charged Off' 'Current' 'Default' 'Fully Paid' 'In Grace Period' 'Issued' 'Late (16-30 days)'
 'Late (31-120 days)']
['0' '1']
['' '36 months' '60 months']
['36' '36' '36' ... '36' '36' '36']


array(['36', '60'], dtype='<U69')

## Manipulating Text Data: Grade and Subgrade

In [13]:
print(np.unique(loan_data_strings[:,3]))
print(np.unique(loan_data_strings[:,4]))
#Filling Subgrade
for i in np.unique(loan_data_strings[:,3])[1:]:
    loan_data_strings[:,4] = np.where((loan_data_strings[:,4] == '') & (loan_data_strings[:,3] == i),
    i + '5',
    loan_data_strings[:,4])

['' 'A' 'B' 'C' 'D' 'E' 'F' 'G']
['' 'A1' 'A2' 'A3' 'A4' 'A5' 'B1' 'B2' 'B3' 'B4' 'B5' 'C1' 'C2' 'C3' 'C4' 'C5' 'D1' 'D2' 'D3' 'D4'
 'D5' 'E1' 'E2' 'E3' 'E4' 'E5' 'F1' 'F2' 'F3' 'F4' 'F5' 'G1' 'G2' 'G3' 'G4' 'G5']


In [14]:
#Removing Grade
loan_data_strings = np.delete(loan_data_strings, 3, axis = 1)
loan_data_strings[:,3]
header_strings = np.delete(header_strings,3)
print(header_strings[3])
#Converting Subgrade
keys = list(np.unique(loan_data_strings[:,3]))
values = list (range(1, np.unique(loan_data_strings[:,3]).shape[0] + 1))
dict_subgrade = dict(zip(keys,values))  

for i in np.unique(loan_data_strings[:,3]):
    loan_data_strings[:,3] = np.where(loan_data_strings[:,3] == i,
                             dict_subgrade[i],
                             loan_data_strings[:,3])
    
np.unique(loan_data_strings[:,3])

sub_grade


array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36',
       '4', '5', '6', '7', '8', '9'], dtype='<U69')

In [15]:
print(np.unique(loan_data_strings[:,4]))

['' 'Not Verified' 'Source Verified' 'Verified']


## Manipulating Text Data: Verification Status

In [16]:
#print(np.unique(loan_data_strings[:,4]))
loan_data_strings[:,4] = np.where((loan_data_strings[:,4]=='') | (loan_data_strings[:,4]=='Not Verified'), 0, 1)
print(np.unique(loan_data_strings[:,4]))

['0' '1']


## Manipulating Text Data: URL

In [17]:
print(loan_data_strings[:,5])


['https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226'
 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261'
 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726' ...
 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990'
 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151'
 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249']


In [18]:
loan_data_strings[:,5] = np.chararray.strip(loan_data_strings[:,5],"https://www.lendingclub.com/browse/loanDetail.action?loan_id=")
print(loan_data_strings[:,5])

['48010226' '57693261' '59432726' ... '50415990' '46154151' '66055249']


In [19]:
print(header_full)

['id' 'issue_d' 'loan_amnt' 'loan_status' 'funded_amnt' 'term' 'int_rate' 'installment' 'grade'
 'sub_grade' 'verification_status' 'url' 'addr_state' 'total_pymnt']


In [20]:
loan_data_numeric[:,0].astype(dtype = np.int32)
loan_data_strings[:,5].astype(dtype = np.int32)
print(loan_data_numeric[:,0])
print(loan_data_strings[:,5])

[48010226. 57693261. 59432726. ... 50415990. 46154151. 66055249.]
['48010226' '57693261' '59432726' ... '50415990' '46154151' '66055249']


In [21]:
np.array_equal(loan_data_numeric[:,0].astype(dtype = np.int32), loan_data_strings[:,5].astype(dtype = np.int32))

True

In [22]:
loan_data_strings = np.delete(loan_data_strings, 5, axis = 1)
header_strings = np.delete(header_strings,5)

In [23]:
print(loan_data_strings[:,5])
print(header_strings)

['CA' 'NY' 'PA' ... 'CA' 'OH' 'IL']
['issue_date' 'loan_status' 'term_months' 'sub_grade' 'verification_status' 'addr_state']


## Manipulating Text Data: State Address

In [24]:
header_strings[5] = "state_address"

In [25]:
np.unique(loan_data_strings[:,5]).size

50

In [26]:
states_names, states_count = np.unique(loan_data_strings[:,5], return_counts = True)
states_count_sorted = np.argsort(-states_count)
states_names[states_count_sorted], states_count[states_count_sorted]

(array(['CA', 'NY', 'TX', 'FL', '', 'IL', 'NJ', 'GA', 'PA', 'OH', 'MI', 'NC', 'VA', 'MD', 'AZ',
        'WA', 'MA', 'CO', 'MO', 'MN', 'IN', 'WI', 'CT', 'TN', 'NV', 'AL', 'LA', 'OR', 'SC', 'KY',
        'KS', 'OK', 'UT', 'AR', 'MS', 'NH', 'NM', 'WV', 'HI', 'RI', 'MT', 'DE', 'DC', 'WY', 'AK',
        'NE', 'SD', 'VT', 'ND', 'ME'], dtype='<U69'),
 array([1336,  777,  758,  690,  500,  389,  341,  321,  320,  312,  267,  261,  242,  222,  220,
         216,  210,  201,  160,  156,  152,  148,  143,  143,  130,  119,  116,  108,  107,   84,
          84,   83,   74,   74,   61,   58,   57,   49,   44,   40,   28,   27,   27,   27,   26,
          25,   24,   17,   16,   10]))

In [27]:
loan_data_strings[:,5] = np.where(loan_data_strings[:,5] == '',
                                  0,
                                  loan_data_strings[:,5])

In [28]:
states_west = np.array(['WA', 'OR','CA','NV','ID','MT', 'WY','UT','CO', 'AZ','NM','HI','AK'])
states_south = np.array(['TX','OK','AR','LA','MS','AL','TN','KY','FL','GA','SC','NC','VA','WV','MD','DE','DC'])
states_midwest = np.array(['ND','SD','NE','KS','MN','IA','MO','WI','IL','IN','MI','OH'])
states_east = np.array(['PA','NY','NJ','CT','MA','VT','NH','ME','RI'])

In [29]:
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_west), 1, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_south), 2, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_midwest), 3, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_east), 4, loan_data_strings[:,5])

In [30]:
np.unique(loan_data_strings[:,5])

array(['0', '1', '2', '3', '4'], dtype='<U69')

## Manipulating Text Data: Converting Strings and Creating a Checkpoint

In [31]:
loan_data_strings

array([['5', '1', '36', '14', '1', '1'],
       ['0', '1', '36', '6', '1', '4'],
       ['9', '1', '36', '11', '1', '4'],
       ...,
       ['6', '1', '36', '6', '1', '1'],
       ['4', '1', '36', '18', '1', '3'],
       ['12', '1', '36', '5', '0', '3']], dtype='<U69')

In [32]:
loan_data_strings = loan_data_strings.astype(np.int)
loan_data_strings

/var/folders/14/kmkw5ndj0kq_53pqzfg6fv2m0000gn/T/ipykernel_35611/648525481.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  loan_data_strings = loan_data_strings.astype(np.int)


array([[ 5,  1, 36, 14,  1,  1],
       [ 0,  1, 36,  6,  1,  4],
       [ 9,  1, 36, 11,  1,  4],
       ...,
       [ 6,  1, 36,  6,  1,  1],
       [ 4,  1, 36, 18,  1,  3],
       [12,  1, 36,  5,  0,  3]])

In [33]:
checkpoint_strings = checkpoint("Checkpoint-Strings",header_strings,loan_data_strings)
checkpoint_strings['header']
checkpoint_strings['data']

array([[ 5,  1, 36, 14,  1,  1],
       [ 0,  1, 36,  6,  1,  4],
       [ 9,  1, 36, 11,  1,  4],
       ...,
       [ 6,  1, 36,  6,  1,  1],
       [ 4,  1, 36, 18,  1,  3],
       [12,  1, 36,  5,  0,  3]])

## Manipulating Numeric Data: Substitute filler values

In [34]:
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

In [35]:
np.isnan(loan_data_numeric).sum()

0

In [36]:
print(header_numeric)

['id' 'loan_amnt' 'funded_amnt' 'int_rate' 'installment' 'total_pymnt']


In [37]:
np.isin(loan_data_numeric[:,0], temporary_fill).sum()

0

In [38]:
print(header_numeric)

['id' 'loan_amnt' 'funded_amnt' 'int_rate' 'installment' 'total_pymnt']


In [39]:
print(temporary_stats)

[[  373332.           nan     1000.           nan     1000.           nan        6.         31.42
          nan         nan         nan         nan         nan        0.  ]
 [54015809.19         nan    15273.46         nan    15311.04         nan       16.62      440.92
          nan         nan         nan         nan         nan     3143.85]
 [68616519.           nan    35000.           nan    35000.           nan       28.99     1372.97
          nan         nan         nan         nan         nan    41913.62]]


In [40]:
print(temporary_stats[:,columns_numeric])

[[  373332.       1000.       1000.          6.         31.42        0.  ]
 [54015809.19    15273.46    15311.04       16.62      440.92     3143.85]
 [68616519.      35000.      35000.         28.99     1372.97    41913.62]]


In [41]:
loan_data_numeric[:,2] = np.where(loan_data_numeric[:,2] == temporary_fill,
                         temporary_stats[0, columns_numeric[2]],
                         loan_data_numeric[:,2])
print(loan_data_numeric[:,2])

[35000. 30000. 15000. ... 10000. 10000. 10000.]


In [42]:
temporary_stats[0,3]

nan

In [43]:
print(temporary_stats[0, columns_numeric[3]])

6.0


In [44]:
print(header_numeric)

for i in [1,3,4,5]:
    loan_data_numeric[:,i] = np.where(loan_data_numeric[:,i] == temporary_fill,
    temporary_stats[2, columns_numeric[i]],
    loan_data_numeric[:,i])
    
print(loan_data_numeric)

['id' 'loan_amnt' 'funded_amnt' 'int_rate' 'installment' 'total_pymnt']
[[48010226.      35000.      35000.         13.33     1184.86     9452.96]
 [57693261.      30000.      30000.         28.99      938.57     4679.7 ]
 [59432726.      15000.      15000.         28.99      494.86     1969.83]
 ...
 [50415990.      10000.      10000.         28.99     1372.97     2185.64]
 [46154151.      35000.      10000.         16.55      354.3      3199.4 ]
 [66055249.      10000.      10000.         28.99      309.97      301.9 ]]


## Manipulating Numeric Data: Currency rate - The exchange rate

In [45]:
EUR_USD = np.genfromtxt(r"EUR-USD.csv", delimiter = ',', autostrip = True, skip_header = 1, usecols = 3)
EUR_USD

array([1.13, 1.12, 1.08, 1.11, 1.1 , 1.12, 1.09, 1.13, 1.13, 1.1 , 1.06, 1.09])

In [46]:
loan_data_strings[:,0]

array([ 5,  0,  9, ...,  6,  4, 12])

In [47]:
exchange_rate = loan_data_strings[:,0]

for i in range(1,13):
    exchange_rate = np.where(exchange_rate == i,
                             EUR_USD[i-1],
                             exchange_rate)

exchange_rate = np.where(exchange_rate == 0, 
                         np.mean(EUR_USD), 
                         exchange_rate)

exchange_rate

array([1.1 , 1.11, 1.13, ..., 1.12, 1.11, 1.09])

In [48]:
exchange_rate.shape

(10000,)

In [49]:
loan_data_numeric.shape

(10000, 6)

In [50]:
exchange_rate = np.reshape(exchange_rate, (10000,1))

In [51]:
loan_data_numeric = np.hstack((loan_data_numeric, exchange_rate))

In [52]:
header_numeric = np.concatenate((header_numeric, np.array(['exchange_rate'])))
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange_rate'],
      dtype='<U19')

## Manipulating Numeric Data: Currency rate - From USD to EUR

In [53]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange_rate'],
      dtype='<U19')

In [54]:
columns_dollar = np.array([1,2,4,5])

In [55]:
loan_data_numeric[:,[columns_dollar]]
loan_data_numeric[:,6]

array([1.1 , 1.11, 1.13, ..., 1.12, 1.11, 1.09])

In [56]:
for i in columns_dollar:
    loan_data_numeric = np.hstack((loan_data_numeric, np.reshape(loan_data_numeric[:,i] / loan_data_numeric[:,6],(10000,1))))

In [57]:
loan_data_numeric.shape

(10000, 11)

In [58]:
header_additional = np.array([column_name + '_EUR' for column_name in header_numeric[columns_dollar]])

In [59]:
header_additional

array(['loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'], dtype='<U15')

In [60]:
header_numeric = np.concatenate((header_numeric, header_additional))

In [61]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange_rate',
       'loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'], dtype='<U19')

In [62]:
header_numeric[columns_dollar] = np.array([column_name + '_USD' for column_name in header_numeric[columns_dollar]])

In [63]:
header_numeric

array(['id', 'loan_amnt_USD', 'funded_amnt_USD', 'int_rate', 'installment_USD', 'total_pymnt_USD',
       'exchange_rate', 'loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'],
      dtype='<U19')

In [64]:
columns_index_order = [0,1,7,2,8,3,4,9,5,10,6]

In [65]:
header_numeric = header_numeric[columns_index_order]

In [66]:
loan_data_numeric = loan_data_numeric[:,columns_index_order]

In [67]:
loan_data_numeric

array([[48010226.  ,    35000.  ,    31933.3 , ...,     9452.96,     8624.69,        1.1 ],
       [57693261.  ,    30000.  ,    27132.46, ...,     4679.7 ,     4232.39,        1.11],
       [59432726.  ,    15000.  ,    13326.3 , ...,     1969.83,     1750.04,        1.13],
       ...,
       [50415990.  ,    10000.  ,     8910.3 , ...,     2185.64,     1947.47,        1.12],
       [46154151.  ,    35000.  ,    31490.9 , ...,     3199.4 ,     2878.63,        1.11],
       [66055249.  ,    10000.  ,     9145.8 , ...,      301.9 ,      276.11,        1.09]])

In [68]:
loan_data_numeric[:,5]

array([13.33, 28.99, 28.99, ..., 28.99, 16.55, 28.99])

In [69]:
loan_data_numeric[:,5] = loan_data_numeric[:,5]/100

In [70]:
loan_data_numeric[:,5]

array([0.13, 0.29, 0.29, ..., 0.29, 0.17, 0.29])

In [71]:
checkpoint_numeric = checkpoint("Checkpoint-Numeric", header_numeric, loan_data_numeric)
checkpoint_numeric['header']
checkpoint_numeric['data']

array([[48010226.  ,    35000.  ,    31933.3 , ...,     9452.96,     8624.69,        1.1 ],
       [57693261.  ,    30000.  ,    27132.46, ...,     4679.7 ,     4232.39,        1.11],
       [59432726.  ,    15000.  ,    13326.3 , ...,     1969.83,     1750.04,        1.13],
       ...,
       [50415990.  ,    10000.  ,     8910.3 , ...,     2185.64,     1947.47,        1.12],
       [46154151.  ,    35000.  ,    31490.9 , ...,     3199.4 ,     2878.63,        1.11],
       [66055249.  ,    10000.  ,     9145.8 , ...,      301.9 ,      276.11,        1.09]])

## Completing the Dataset

In [72]:
checkpoint_strings['data'].shape

(10000, 6)

In [73]:
checkpoint_numeric['data'].shape

(10000, 11)

In [74]:
np.hstack((checkpoint_numeric['data'], checkpoint_strings['data'])).shape

(10000, 17)

In [75]:
loan_data = np.hstack((checkpoint_numeric['data'], checkpoint_strings['data']))

In [76]:
loan_data

array([[48010226.  ,    35000.  ,    31933.3 , ...,       14.  ,        1.  ,        1.  ],
       [57693261.  ,    30000.  ,    27132.46, ...,        6.  ,        1.  ,        4.  ],
       [59432726.  ,    15000.  ,    13326.3 , ...,       11.  ,        1.  ,        4.  ],
       ...,
       [50415990.  ,    10000.  ,     8910.3 , ...,        6.  ,        1.  ,        1.  ],
       [46154151.  ,    35000.  ,    31490.9 , ...,       18.  ,        1.  ,        3.  ],
       [66055249.  ,    10000.  ,     9145.8 , ...,        5.  ,        0.  ,        3.  ]])

In [77]:
np.isnan(loan_data).sum()

0

In [78]:
np.concatenate((checkpoint_numeric['header'], checkpoint_strings['header']))

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange_rate',
       'issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'state_address'], dtype='<U19')

In [79]:
header_full = np.concatenate((checkpoint_numeric['header'], checkpoint_strings['header']))

In [80]:
loan_data = loan_data[np.argsort(loan_data[:,0])]

In [81]:
np.argsort(loan_data[:,0])

array([   0,    1,    2, ..., 9997, 9998, 9999])

## Storing the new Dataset

In [82]:
loan_data = np.vstack((header_full, loan_data))
np.savetxt("Loan-Data-Preprocessed.csv", loan_data, fmt = '%s',delimiter = ',')
